In [42]:
import pandas as pd
df = pd.read_csv('ssbpts.csv')

df2 = df# df.query('MidLat > 30 and MidLat < 75 and MidLon < -60 and MidLon > -135 ')

try:
    dfe = pd.read_csv('errors.csv', index_col=0)
    df3 = pd.concat([df2, dfe], axis = 1)
    df4 = df3.query('Status == "OK"')
except FileNotFoundError:
    dfe = pd.DataFrame()
    df4 = df2

In [44]:
import obspy
from obspy.signal.filter import envelope
from scipy.signal import hilbert

from numpy import argmax, zeros

rbin = zeros(200)

snr ={}

for row in df4.iterrows():
    bpindex = row[0]
    filename = 'data/mseed/%012d.mseed' % bpindex
    st = obspy.read(filename).detrend().taper(0.05).filter('lowpass',freq=0.1,corners=4)
    
    tr = st[0]
    
    noise   = tr.copy().trim(tr.stats.starttime, tr.stats.starttime+100)
    signal  = tr.copy().trim(tr.stats.starttime+100, tr.stats.starttime+200)
    
    if len(noise.data) == 0 or len(signal.data) == 0:
        snr[bpindex] = -1.0
        continue
    
    snr[bpindex]    = max(abs(envelope(signal.data))) / max(abs(envelope(noise.data)))
    
    if row[1]["MidLat"] < 30:
        print("STOP")
        break
    

    

/Users/mancinelli/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in double_scalars


STOP


In [38]:
snr.items()
dfout = pd.DataFrame(snr, index=["SNR"]).T

In [39]:
dfout.to_csv('snr.csv')

In [40]:
dfout.query("SNR > 3.0")

,SNR
2301,3.074457
4581,3.286390
5709,4.271263
11379,3.588503
11382,3.897003
11397,6.295181
11424,3.594863
12116,3.742650
12855,3.693870
31268,3.120215


In [47]:
df4.query('MidLat < 30')

,Event,Network,Station,Range,Azimuth,BAzimuth,EvtLat,EvtLon,StaLat,StaLon,MidLat,MidLon,EvtTime,Status
427191,C201407132000A,AZ,FRD,94.118675,56.755125,267.904048,-4.08,151.29,33.494700,-116.602200,20.600003,-167.860929,2014-07-13T20:00:39.600000Z,OK
427517,C201407132000A,CU,SDDR,136.097854,66.675970,284.465046,-4.08,151.29,18.982100,-71.287800,19.858581,-143.811434,2014-07-13T20:00:39.600000Z,OK
427629,C201407132000A,GS,OK027,109.869423,53.459656,279.638140,-4.08,151.29,35.711956,-97.283642,26.436994,-161.452285,2014-07-13T20:00:39.600000Z,OK
427630,C201407132000A,GS,OK028,110.073352,53.585263,279.724853,-4.08,151.29,35.561110,-97.061389,26.390239,-161.299216,2014-07-13T20:00:39.600000Z,OK
427631,C201407132000A,GS,OK029,109.717914,53.395793,279.559660,-4.08,151.29,35.796570,-97.454860,26.449898,-161.554048,2014-07-13T20:00:39.600000Z,OK
427636,C201407132000A,IC,KMI,55.432499,304.324814,114.591026,-4.08,151.29,25.123300,102.740000,11.455576,128.216835,2014-07-13T20:00:39.600000Z,OK
427638,C201407132000A,IC,WMQ,74.282321,317.685354,111.705877,-4.08,151.29,43.813801,87.704903,22.868846,125.113279,2014-07-13T20:00:39.600000Z,OK
427641,C201407132000A,II,ABPO,102.074695,249.598531,98.682753,-4.08,151.29,-19.018000,47.229000,-18.367193,101.124153,2014-07-13T20:00:39.600000Z,OK
427650,C201407132000A,II,EFI,119.086296,159.700990,213.840478,-4.08,151.29,-51.675300,-58.063700,-57.404184,-174.991003,2014-07-13T20:00:39.600000Z,OK
427680,C201407132000A,IU,CASY,68.306922,196.485605,44.560521,-4.08,151.29,-66.279200,110.535400,-36.573015,139.848080,2014-07-13T20:00:39.600000Z,OK


In [48]:
dfe

,Status
2197,OK
2300,OK
2301,OK
2302,OK
2305,OK
2307,OK
2309,OK
2312,No Data at II.ALE
2321,OK
2342,OK
